In [1]:
from sklearn.ensemble import (AdaBoostRegressor, 
                              BaggingRegressor, 
                              ExtraTreesRegressor, 
                              GradientBoostingRegressor, 
                              RandomForestRegressor)

import pandas as pd
import custom_funcs as cf

%load_ext autoreload
%autoreload 2
%matplotlib inline

# Purpose

In this notebook, I intend to setup an automated ensemble learning model selection procedure for any given drug.

In [9]:
data_protease = pd.read_csv('../data/hiv-protease-data.csv', index_col='SeqID')
protease_drug_cols = data_protease.columns[0:8]
protease_feat_cols = data_protease.columns[8:]

data_nnrt = pd.read_csv('../data/hiv-nnrt-data.csv', index_col='SeqID')
nnrt_drug_cols = data_nnrt.columns[0:4]
nnrt_feat_cols = data_nnrt.columns[4:]

data_nrt = pd.read_csv('../data/hiv-nrt-data.csv', index_col='SeqID')
nrt_drug_cols = data_nrt.columns[0:6]
nrt_feat_cols = data_nrt.columns[6:]

In [13]:
data_nrt.shape

(1498, 246)

In [19]:
df_nnrt = cf.replace_ambiguous_letters_with_nan(data_nnrt, nnrt_feat_cols)
df_nnrt = cf.replace_dashes_with_canonical_letters(df_nnrt, rt_consensus, nnrt_feat_cols)
# df_nnrt.dropna(subset=nnrt_feat_cols).shape
# df_nnrt.shape

(570, 244)

In [15]:
rt_consensus = cf.read_consensus('../data/hiv-rt-consensus.fasta')
protease_consensus = cf.read_consensus('../data/hiv-protease-consensus.fasta')

In [23]:
import numpy as np
df_nnrt.dropna().isnull().values.any()

False